# Cleaning Election Data for Tableau Dashboard

This notebook contains the script used to clean the dataset of election results for the 2020 U.S. Presidential Election.

The original dataset from Reuters and posted to Kaggle by Raphael Fontes can be found [here](https://www.kaggle.com/unanimad/us-election-2020?select=president_county_candidate.csv).

The Tableau dashboard created with this data can also be found [here](https://public.tableau.com/profile/tyler.dangel#!/vizhome/ElectionData_16091568606920/ElectionData)

In [1]:
#Import necessary packages
import pandas as pd
import numpy as np

#Read in .csv file
df = pd.read_csv("president_county_candidate.csv")

In [2]:
df.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [3]:
print(df.shape)
df.isnull().sum()

(31902, 6)


state          0
county         0
candidate      0
party          0
total_votes    0
won            0
dtype: int64

In [4]:
#Drop rows containing 0 for total_votes
df.drop(inplace=True, index=df[df["total_votes"] == 0].index)

#Remove white space on candidate
df["candidate"] = df["candidate"].str.strip()

## Removing Low-share Candidates

The first major step in the cleaning process is to clean up rows for candidates with very small vote totals. There are 38 distinct candidates in this dataset and 98% of the total vote is attributed to two candidates, Joe Biden and Donald Trump. Reassigning the 34 candidates with the lowest turnout to "Other" will make the data more mangageable for visualizations while leaving five possible candidate entries for some exploration.

In [5]:
#Create pivot table and then normalize values
candidate_frequency = pd.pivot_table(df, values="total_votes", index="candidate", aggfunc=np.sum)/df["total_votes"].sum() * 100

#Move index to a Series for merging with original DF
candidate_frequency.reset_index(inplace=True)

#Rename values column to total_percentage
candidate_frequency = candidate_frequency.rename(columns={"total_votes":"total_percentage"})

#Sort pivot table to retrieve top 4 candidates
candidate_frequency.sort_values("total_percentage", ascending=False, inplace=True)

#Define vote percentage of 4th candidate as minimum for conditional later
minimum_percentage = candidate_frequency.iloc[3, 1]

In [6]:
#Merge total_percentage into the main DataFrame for use later
df_merged = df.merge(candidate_frequency, how="left", on="candidate")

df_merged

,state,county,candidate,party,total_votes,won,total_percentage
0,Delaware,Kent County,Joe Biden,DEM,44552,True,51.421263
1,Delaware,Kent County,Donald Trump,REP,41009,False,46.744242
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False,1.174577
3,Delaware,Kent County,Howie Hawkins,GRN,420,False,0.253736
4,Delaware,New Castle County,Joe Biden,DEM,195034,True,51.421263
...,...,...,...,...,...,...,...
27130,Arizona,Maricopa County,Write-ins,WRI,1331,False,0.128670
27131,Arizona,Mohave County,Donald Trump,REP,78535,True,46.744242
27132,Arizona,Mohave County,Joe Biden,DEM,24831,False,51.421263
27133,Arizona,Mohave County,Jo Jorgensen,LIB,1302,False,1.174577


In [7]:
#Create list for iterating over candidates with low vote percentage
index = df_merged[df_merged["total_percentage"] < minimum_percentage].index.tolist()

#Iterate over merged DataFrame to replace candidate names and parties
for i in index:
    df_merged.iloc[i, 2] = "Other"
    df_merged.iloc[i, 3] = "N/A"

In [8]:
print(df_merged["candidate"].value_counts())

Other            10461
Joe Biden         4630
Donald Trump      4629
Jo Jorgensen      4565
Howie Hawkins     2850
Name: candidate, dtype: int64


In [9]:
#Remove percentage column
df_merged.drop(columns="total_percentage", inplace=True)

## Aggregating the Data

Now that the candidates have been reassigned, the data will need to be aggregated to combine all of the rows with the candidate of "Other".

In [10]:
#Create DataFrame for rows with "Other" candidate
other_df = df_merged[df_merged["candidate"] == "Other"]

#Determine total rows of "Other"
other_total = other_df["county"].count()

#Determine unique counties with "Other"
other_remain = other_df[["county", "state"]].drop_duplicates()["county"].count()

#Calculate what the resulting number of rows sould be after aggregating data
row_target = df_merged["county"].count() - (other_total - other_remain)

In [11]:
#Aggregate DataFrame to consolidate "Other" rows
df_grouped = df_merged.groupby(["state", "county", "candidate"]).aggregate({"party":"first",
                                                                            "total_votes":"sum",
                                                                            "won":"first"
                                                                            }
                                                                           )

df_grouped.reset_index(inplace=True)

In [12]:
#Check if row count matches target
row_count = df_grouped["county"].count()

if row_count == row_target:
    print("DataFrame aggregated successfully. {:,} rows in resulting DataFrame.".format(row_count))
else:
    print("DataFrame not aggregated successfully. {:,} expected rows vs. {:,} output rows".format(row_target, row_count))

DataFrame aggregated successfully. 20,014 rows in resulting DataFrame.


In [13]:
#Aggregate data to determine total votes per county
df_county_totals = df_grouped.groupby(["state", "county"]).agg({"total_votes":"sum"})

#Merge newly aggregated data with original dataset
df_grouped = pd.merge(df_grouped, df_county_totals, how="left", on=["state", "county"])

In [14]:
df_grouped.head()

,state,county,candidate,party,total_votes_x,won,total_votes_y
0,Alabama,Autauga County,Donald Trump,REP,19838,True,27770
1,Alabama,Autauga County,Jo Jorgensen,LIB,350,False,27770
2,Alabama,Autauga County,Joe Biden,DEM,7503,False,27770
3,Alabama,Autauga County,Other,N/A,79,False,27770
4,Alabama,Baldwin County,Donald Trump,REP,83544,True,109679


In [15]:
#Use new column to determine percentage of vote received by each candidate
df_grouped["percentage"] = df_grouped["total_votes_x"]/df_grouped["total_votes_y"]

#Rename original votes column
df_grouped.rename(columns={"total_votes_x":"total_votes"}, inplace=True)

#Drop unnecessary votes column
df_grouped.drop(["total_votes_y"], axis=1, inplace=True)

In [16]:
df_grouped.head()

,state,county,candidate,party,total_votes,won,percentage
0,Alabama,Autauga County,Donald Trump,REP,19838,True,0.714368
1,Alabama,Autauga County,Jo Jorgensen,LIB,350,False,0.012604
2,Alabama,Autauga County,Joe Biden,DEM,7503,False,0.270184
3,Alabama,Autauga County,Other,N/A,79,False,0.002845
4,Alabama,Baldwin County,Donald Trump,REP,83544,True,0.761714


## State-level Aggregation

With the county-level data cleaned, I can continue to clean the data down to the state-level. I will create two new DataFrames for this view, one containing total turnout for each state and the other containing only the winner and vote turnout.

In [17]:
#Aggregate DataFrame to create table for state results
df_state = df_grouped.groupby(["state", "candidate"]).agg({"party":"first",
                                                         "total_votes" : "sum",
                                                         }
                                                        )
#Reset index for consistent formatting
df_state.reset_index(inplace=True)

In [18]:
df_state_winner = df_state.copy()

#Create new duplicate column of total_votes
df_state_winner["total_votes_sum"] = df_state_winner["total_votes"].copy()

#Aggregate df_state to the winning vote count of each state
df_state_winner = df_state_winner.groupby(["state"]).agg({"total_votes":"max", "total_votes_sum":"sum"})

#Clean index
df_state_winner.reset_index(inplace=True)

#Merge new DataFrame with df_state to include winner's name
df_state_winner = pd.merge(df_state_winner, df_state[["state", "total_votes", "candidate"]], how="left", on=["state", "total_votes"] )

if df_state_winner.shape[0] == 51:
    print("Aggregation successful. Output contains {} state results.".format(df_state_winner.shape[0]))

Aggregation successful. Output contains 51 state results.


In [19]:
#Rename columns of df_state_winner
df_state_winner.columns = ["state", "winner_votes", "total_votes", "winner"]

#Add vote percentage of winner to the DataFrame
df_state_winner["percentage"] = df_state_winner["winner_votes"]/df_state_winner["total_votes"]

### Electoral Votes

Another useful piece of information for this data is the number of electoral college votes each state has. The CSV file used can be found [here](https://worldpopulationreview.com/state-rankings/electoral-votes-by-state).

In [20]:
#Read in CSV file containing the electoral votes for eaech state
df_college = pd.read_csv("electoral_college.csv")

#Rename columns for easier merging
df_college.columns = ["state", "ec_votes", "people_per_vote", "population"]

#Add electoral college votes to the df_state_winner DataFrame
df_state_winner = pd.merge(df_state_winner, df_college[["state", "ec_votes"]], how="left", on="state")

In [21]:
#The file with the electoral college was missing WAshington D.C., so that will need added in
index_key = df_state_winner.index[df_state_winner["state"] == "District of Columbia"].tolist()

#D.C. has 3 votes, and it will be easiest just to enter this manually
df_state_winner.loc[index_key, "ec_votes"] = 3

## Exporting Data

Finally, I will export the datasets to CSV files for use in Tableau

In [22]:
#Export DataFrames to new .csv files for use in Tableau
df_grouped.to_csv("clean_data/president_county_candidate_clean.csv", index=False)

df_state.to_csv("clean_data/president_state_clean.csv", index=False)

df_state_winner.to_csv("clean_data/president_state_winner_clean.csv", index=False)